In [139]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from Pymoe import Kitsu
import wikipedia
import time
from Pymoe import Anilist
import html

## Scraping

In [26]:
webUrl = requests.get('https://bayart.org/anime-quotes/')
webData = BeautifulSoup(web_url.content,'lxml')

In [27]:
quoteData = web_data.findAll('li')
quotes = []
for quote in quoteData:
    quoteText = quote.get_text()
    if len(quoteText)>0:
        if quoteText[0] == '“':
            quotes.append(quoteText.replace('\xa0',''))

## Cleaning

In [30]:
quotedf = pd.DataFrame(quotes, columns=['Quote'])
quotedf.head()

,Quote
0,“It’s just pathetic to give up on something be...
1,“The world isn’t perfect. But it’s there for u...
2,"“If you win, you live. If you lose, you die. I..."
3,“Weaklings will stay weak forever. But weaknes...
4,“I’m not stupid. I’m just too lazy to show how...


In [40]:
dasheddf = quotedf['Quote'].str.split('–', expand=True).dropna()
# change to loop over everything except last col eventually
dasheddf['Quote'] = dasheddf[0] + "-" + dasheddf[1]
dasheddf['Author'] = dasheddf[2]
dasheddf = dasheddf[['Quote', 'Author']]
dasheddf

,Quote,Author
48,“Thinking you’re no-good and worthless is the ...,Nobito
81,"“In this world, wherever there is light - ther...",Madara Uchiha
99,“Love is the reason why there is a pain. When ...,Pain


In [47]:
undasheddf = quotedf['Quote'].str.split('–', expand=True)
undasheddf = undasheddf[undasheddf[2].isnull()]
undasheddf = undasheddf.drop([2], axis=1)
undasheddf = undasheddf.rename({0: 'Quote', 1: 'Author'}, axis=1)
undasheddf.head()

,Quote,Author
0,“It’s just pathetic to give up on something be...,Reiko Mikami
1,“The world isn’t perfect. But it’s there for u...,Roy Mustang
2,"“If you win, you live. If you lose, you die. I...",Shingeki No Kyojin
3,“Weaklings will stay weak forever. But weaknes...,Makarov Dreyar
4,“I’m not stupid. I’m just too lazy to show how...,Oreki Houtarou


In [50]:
quotedf = undasheddf.append(dasheddf)
quotedf['Quote'] = quotedf['Quote'].str.replace('“','')
quotedf['Quote'] = quotedf['Quote'].str.replace('”','')
quotedf.head()

,Quote,Author
0,It’s just pathetic to give up on something bef...,Reiko Mikami
1,The world isn’t perfect. But it’s there for us...,Roy Mustang
2,"If you win, you live. If you lose, you die. If...",Shingeki No Kyojin
3,Weaklings will stay weak forever. But weakness...,Makarov Dreyar
4,I’m not stupid. I’m just too lazy to show how ...,Oreki Houtarou


## Adding extra data

In [148]:
authors = quotedf['Author'].values
instance = Anilist()

In [155]:
def searchChar(author):
    result = {'description':'', 'image':''}
    charBaseData = instance.search.character(author)['data']['Page']['characters']
    if len(charBaseData) > 0:
        charID = charBaseData[0]['id']
        charData = instance.get.character(charID)
        charDescription = charData['data']['Character']['description']
        if charDescription is not None:
            result['description'] = html.unescape(charDescription).replace('!~','').replace('~!','')
        # using html to get rid of unconverted html symbols
        result['image'] = charData['data']['Character']['image']['large']
    else:
        animeBaseData = instance.search.anime(author)['data']['Page']['media']
        if len(animeBaseData) > 0:
            animeID = animeBaseData[0]['id']
            animeData = instance.get.anime(animeID)
            animeDescription = animeData['data']['Media']['description']
            if animeDescription is not None:
                result['description'] = html.unescape(animeDescription).replace('!~','').replace('~!','')
            result['image'] = animeData['data']['Media']['coverImage']['large']
            # we can also use ['bannerImage'] instead of ['coverImage']['large'] for different pictures
    return result

In [156]:
summaries = []
images = []
count = 0
for author in authors:
    charJson = searchChar(author)
    summaries.append(charJson['description'])
    images.append(charJson['image'])
    count = count + 1
    if (count == 40):
        # rate limited to 90 per min and each run takes 2 calls
        time.sleep(55)
        count = 0

In [159]:
quotedf['Summary'] = summaries
quotedf['Image'] = images
quotedf.head()

,Quote,Author,Summary,Image
0,It’s just pathetic to give up on something bef...,Reiko Mikami,Kouichi's aunt from Ritsuko's side and class ...,https://s4.anilist.co/file/anilistcdn/characte...
1,The world isn’t perfect. But it’s there for us...,Roy Mustang,__Birthday:__ 1885 (Age 30 from vol. 10)\n__He...,https://s4.anilist.co/file/anilistcdn/characte...
2,"If you win, you live. If you lose, you die. If...",Shingeki No Kyojin,"Several hundred years ago, humans were nearly ...",https://s4.anilist.co/file/anilistcdn/media/an...
3,Weaklings will stay weak forever. But weakness...,Makarov Dreyar,"Makarov (マカロフ Makarofu), is the ""Master"" of th...",https://s4.anilist.co/file/anilistcdn/characte...
4,I’m not stupid. I’m just too lazy to show how ...,Oreki Houtarou,__Age:__ 16-17 \n__Height:__ 176 cm \n__Birthd...,https://s4.anilist.co/file/anilistcdn/characte...


In [167]:
finalquotedf = quotedf[(quotedf['Summary'] != '') & (quotedf['Image'] != '')]

In [171]:
finalquotedf.to_json('animequotes.json', orient="records")